In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import time
from datetime import datetime

# --- PARÁMETROS ESPECÍFICOS PARA TU TAREA ---
GENRES = ['latin']
YEARS = [2010, 2024] 

In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')

In [4]:
# Años para iterar
YEARS = [2010, 2024] 

# Términos optimizados para una búsqueda de ALTO VOLUMEN
# Incluye etiquetas 
query_terms = [
    'pop',
    'latin pop',
    'spanish pop',
    'pop latino',
    'soft latin'
    # Entre comillas para buscar la frase exacta (útil para el 2010)
]

# 4. Inicializa el cliente de Spotify
try:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")
    sp = None

✅ Conexión a Spotify exitosa.


In [5]:
# NUEVA CELDA: Comprobación de autenticación
print(type(sp)) 

<class 'spotipy.client.Spotify'>


In [6]:
# --- CELDA 2: DEFINICIÓN DE LA FUNCIÓN ESTABLE CON DOBLE LÍMITE (500 TOTAL, 250 ANUAL) ---
import time
import pandas as pd
def extract_spotify_data_simplificada(sp, years, query_terms):
    """
    VERSION ESTABLE CON LÍMITES: 500 total, 250 por año. Extrae datos básicos
    y la Popularidad (sin Audio Features) para garantizar estabilidad.
    """
    if sp is None:
        print("Error: Cliente de Spotify no inicializado.")
        return pd.DataFrame()
    spotify_data = []
    MAX_TOTAL_SONGS = 500     # Límite global
    MAX_SONGS_PER_YEAR = 250  # Límite por año (REINTRODUCIDO)
    for year in years:
        # 1. VERIFICACIÓN DEL LÍMITE GLOBAL
        if len(spotify_data) >= MAX_TOTAL_SONGS:
            print(f":señal_octogonal: LÍMITE GLOBAL ALCANZADO ({MAX_TOTAL_SONGS}). Terminando la extracción.")
            break
        year_range = f'{year}-{year}'
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")
        songs_added_in_year = 0 # Contador reiniciado para el límite anual
        for term in query_terms:
            # 2. VERIFICACIÓN DEL LÍMITE ANUAL (antes de la búsqueda)
            if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                print(f":señal_octogonal: LÍMITE ANUAL ALCANZADO ({MAX_SONGS_PER_YEAR}) para {year}. Pasando al siguiente año.")
                break # Rompe el bucle de términos, y avanza al siguiente año.
            query = f'{term} year:{year_range}'
            print(f"   -> Buscando con término: {term}")
            # --- BLOQUE DE BÚSQUEDA ---
            try:
                results = sp.search(q=query, type='track', limit=50)
            except Exception as e:
                print(f"   :x: ERROR grave al buscar con '{term}': {e}. Esperando 5s.")
                time.sleep(5)
                continue
            songs_added_in_term = 0
            while results:
                for track in results['tracks']['items']:
                    track_id = track['id']
                    # 3. VERIFICACIÓN DE LÍMITES Y DEDUPLICACIÓN
                    if len(spotify_data) >= MAX_TOTAL_SONGS:
                        print(f":señal_octogonal: LÍMITE GLOBAL ALCANZADO ({MAX_TOTAL_SONGS}). Terminando la extracción.")
                        return pd.DataFrame(spotify_data)
                    # 4. VERIFICACIÓN DE LÍMITE ANUAL (por si se alcanza con la canción actual)
                    if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                        break # Rompe el bucle de canciones (for track...), pasará al chequeo de abajo.
                    if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                        continue
                    # 5. Añadir la canción
                    data_row = {
                        'Artista': track['artists'][0]['name'],
                        'Género musical': f'Latin (Origen: {term})',
                        'Tipo': 'Canción',
                        'Nombre': track['name'],
                        'Año de lanzamiento': year,
                        'ID_Spotify': track_id,
                        'ID_Album': track['album']['id'],
                        'Popularidad': track.get('popularity', None)
                    }
                    spotify_data.append(data_row)
                    songs_added_in_year += 1
                    songs_added_in_term += 1
                # Rompe el bucle de paginación si se alcanzó el límite anual en el bucle interior.
                if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                    break
                # 6. Paginación
                try:
                    if results['tracks']['next']:
                        results = sp.next(results['tracks'])
                        time.sleep(1.0) # Pausa estable entre páginas de 50
                    else:
                        results = None
                except Exception as e:
                    print(f"   :x: ERROR al obtener la página siguiente: {e}")
                    results = None
            print(f"      -> Añadidas {songs_added_in_term} canciones ÚNICAS de '{term}'.")
    return pd.DataFrame(spotify_data)

In [10]:
# --- CELDA 3: LLAMADA, EJECUCIÓN Y GUARDADO DEL CSV ---
if sp:
    # Llamamos a la versión simplificada y estable
    df_spotify_raw = extract_spotify_data_simplificada(sp, YEARS, query_terms)
    print("\n--- RESULTADO FINAL DE EXTRACCIÓN ESTABLE ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)}")
    print(df_spotify_raw[['Nombre', 'Artista', 'Popularidad']].head())
    # Guardar el nuevo CSV (¡Importante para la Consolidación!)
    file_output = 'reggaeton_data_2010_2024.csv'
    df_spotify_raw.to_csv(file_output, index=False)
    print(f":marca_de_verificación_blanca: Datos guardados en: {file_output}")
else:
    print(":x: El cliente 'sp' no está inicializado. No se pudo ejecutar la extracción.")


--- INICIANDO EXTRACCIÓN PARA EL AÑO 2010 ---
   -> Buscando con término: pop
      -> Añadidas 41 canciones ÚNICAS de 'pop'.
   -> Buscando con término: latin pop
      -> Añadidas 89 canciones ÚNICAS de 'latin pop'.
   -> Buscando con término: spanish pop
      -> Añadidas 99 canciones ÚNICAS de 'spanish pop'.
   -> Buscando con término: pop latino
      -> Añadidas 21 canciones ÚNICAS de 'pop latino'.
:señal_octogonal: LÍMITE ANUAL ALCANZADO (250) para 2010. Pasando al siguiente año.

--- INICIANDO EXTRACCIÓN PARA EL AÑO 2024 ---
   -> Buscando con término: pop
      -> Añadidas 25 canciones ÚNICAS de 'pop'.
   -> Buscando con término: latin pop
      -> Añadidas 96 canciones ÚNICAS de 'latin pop'.
   -> Buscando con término: spanish pop
      -> Añadidas 100 canciones ÚNICAS de 'spanish pop'.
   -> Buscando con término: pop latino
:señal_octogonal: LÍMITE GLOBAL ALCANZADO (500). Terminando la extracción.

--- RESULTADO FINAL DE EXTRACCIÓN ESTABLE ---
Total de registros extraídos: 

In [11]:
# VERIFICACIÓN DE CANCIONES
print(f"Número de canciones encontradas en df_spotify_raw: {len(df_spotify_raw)}")
print(f"Primeras filas (head) de df_spotify_raw:\n{df_spotify_raw.head()}")

Número de canciones encontradas en df_spotify_raw: 500
Primeras filas (head) de df_spotify_raw:
        Artista       Género musical     Tipo  \
0   Billy Ocean  Latin (Origen: pop)  Canción   
1   Nicki Minaj  Latin (Origen: pop)  Canción   
2  David Guetta  Latin (Origen: pop)  Canción   
3  Joey Montana  Latin (Origen: pop)  Canción   
4       Pitbull  Latin (Origen: pop)  Canción   

                                      Nombre  Año de lanzamiento  \
0  Caribbean Queen (No More Love On the Run)                2010   
1                                 Super Bass                2010   
2                  Memories (feat. Kid Cudi)                2010   
3                                 La Melodía                2010   
4                   Guantanamera (She's Hot)                2010   

               ID_Spotify                ID_Album  Popularidad  
0  4JEylZNW8SbO4zUyfVrpb7  7n4OT3zEZaEiyKKd6mFAhi           75  
1  3hlksXnvbKogFdPbpO9vel  7aADdYLiK1z7GlMFr0UIZw           80  
2  7f

In [12]:
# --- CELDA COMPLETA: EXTRACCIÓN Y UNIFICACIÓN DE ÁLBUMES ---

# Asegúrate de que las librerías necesarias (pandas, time) y la variable 'sp' estén disponibles
import pandas as pd
import time
# Nota: La variable 'sp' debe estar definida por la Celda 1.

# -------------------------------------------------------------------
# BLOQUE 1: DEFINICIÓN DE LA FUNCIÓN
# -------------------------------------------------------------------

def extract_album_details(sp, album_ids):
    """
    Extrae los metadatos completos (nombre, año, artista) de los álbumes.
    """
    album_details = []
    
    # El set ya se crea cuando se llama a la función con album_ids_list.
    unique_album_ids = album_ids
    
    print(f"\n--- INICIANDO EXTRACCIÓN DE METADATOS DE {len(unique_album_ids)} ÁLBUMES ---")
    
    for album_id in unique_album_ids:
        try:
            album_info = sp.album(album_id) 
            
            data_row = {
                'ID_Album': album_id,
                'Nombre_Album': album_info['name'],
                'Año_Lanzamiento_Album': album_info['release_date'][:4], # Tomamos solo el año
                'Artista_Principal_Album': album_info['artists'][0]['name'],
                'Tipo_Lanzamiento': album_info['album_type'] 
            }
            album_details.append(data_row)
            time.sleep(0.2) # Pausa para no saturar la API
            
        except Exception as e:
            # Capturamos errores como álbumes que han sido eliminados de Spotify
            print(f"   -> Error al extraer detalles del álbum {album_id}: {e}")
            pass
            
    print(f"✅ Extracción de {len(album_details)} álbumes completada.")
    return pd.DataFrame(album_details)

# -------------------------------------------------------------------
# BLOQUE 2: EJECUCIÓN Y FUSIÓN
# -------------------------------------------------------------------

# Verificación de que el DataFrame de canciones está disponible
if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    print("\n--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---")
    
    # 1. VERIFICACIÓN CRÍTICA: La columna 'ID_Album' debe existir.
    if 'ID_Album' not in df_spotify_raw.columns:
        print("❌ ERROR CRÍTICO: La columna 'ID_Album' NO se encontró. La Celda 3 falló al guardar el ID del álbum.")
    else:
        
        # 2. Obtener IDs únicos válidos de los álbumes
        album_ids_list = df_spotify_raw['ID_Album'].dropna().unique() 
        
        if len(album_ids_list) == 0:
            print("❌ LISTA VACÍA: Se encontraron canciones, pero 0 IDs de álbumes válidos. No se puede continuar.")
        else:
            print(f"✅ Se encontraron {len(album_ids_list)} IDs de álbumes únicos. Extrayendo ahora...")
            
            # 3. Ejecutar la función de extracción (Tardará unos minutos)
            df_album_details = extract_album_details(sp, album_ids_list)
            
            # 4. Fusionar (JOIN) los detalles del álbum con los datos de las canciones
            df_spotify_final = pd.merge(
                df_spotify_raw,
                df_album_details[['ID_Album', 'Nombre_Album', 'Año_Lanzamiento_Album', 'Artista_Principal_Album']],
                on='ID_Album',
                how='left' 
            )
            
            print(f"\n--- PROCESO DE ÁLBUMES COMPLETADO ---")
            print(f"DataFrame intermedio 'df_spotify_final' creado con {len(df_spotify_final)} filas.")
            print(df_spotify_final.head())
            
else:
    print("❌ ERROR: El DataFrame de canciones (df_spotify_raw) no está disponible. Vuelve a ejecutar la Celda 3.")


--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---
✅ Se encontraron 334 IDs de álbumes únicos. Extrayendo ahora...

--- INICIANDO EXTRACCIÓN DE METADATOS DE 334 ÁLBUMES ---
✅ Extracción de 334 álbumes completada.

--- PROCESO DE ÁLBUMES COMPLETADO ---
DataFrame intermedio 'df_spotify_final' creado con 500 filas.
        Artista       Género musical     Tipo  \
0   Billy Ocean  Latin (Origen: pop)  Canción   
1   Nicki Minaj  Latin (Origen: pop)  Canción   
2  David Guetta  Latin (Origen: pop)  Canción   
3  Joey Montana  Latin (Origen: pop)  Canción   
4       Pitbull  Latin (Origen: pop)  Canción   

                                      Nombre  Año de lanzamiento  \
0  Caribbean Queen (No More Love On the Run)                2010   
1                                 Super Bass                2010   
2                  Memories (feat. Kid Cudi)                2010   
3                                 La Melodía                2010   
4                   Guantanamera (She's Hot)     

In [13]:
def extract_lastfm_artist_details(artist_name):
    """Extrae biografía y estadísticas de popularidad de un artista."""
    
    url = 'http://ws.audioscrobbler.com/2.0/'
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': LASTFM_API_KEY,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'artist' in data:
            artist_info = data['artist']
            
            # Extraer los datos relevantes (Biografía, Popularidad/Plays)
            biography = artist_info.get('bio', {}).get('summary', 'N/A')
            playcount = artist_info.get('stats', {}).get('playcount', '0')
            listeners = artist_info.get('stats', {}).get('listeners', '0')
            
            return {
                'Artista': artist_name,
                'Biografia_Resumen': biography,
                'Playcount_LastFM': playcount,
                'Listeners_LastFM': listeners,
            }
    except Exception:
        # En caso de error de conexión o artista no encontrado, devuelve None
        pass 
        
    return None

# --- Ejecución para Artistas Únicos ---

if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    # Obtener la lista única de artistas para evitar búsquedas repetidas
    unique_artists = df_spotify_raw['Artista'].unique()
    lastfm_results = []
    
    print("\n--- EXTRACCIÓN DETALLES LAST.FM ---")
    
    for artist in unique_artists: 
        details = extract_lastfm_artist_details(artist)
        if details:
            lastfm_results.append(details)
        time.sleep(0.5) # Pausa crucial para no sobrecargar la API

    df_lastfm = pd.DataFrame(lastfm_results)
    
    # 5. UNIFICACIÓN DE DATOS
    # Fusionar los datos de Spotify con los detalles de Last.fm
    df_final_latin = pd.merge(
        df_spotify_raw, 
        df_lastfm[['Artista', 'Playcount_LastFM', 'Listeners_LastFM']],
        on='Artista', 
        how='left' # Usamos left join para mantener todas las canciones de Spotify
    )
    
    print("\n--- DATAFRAME FINAL DE REGGAETON LISTO PARA ALMACENAMIENTO ---")
    print(df_final_latin.head())
    print(f"Total de registros finales para Latin: {len(df_final_latin)}")


--- EXTRACCIÓN DETALLES LAST.FM ---

--- DATAFRAME FINAL DE REGGAETON LISTO PARA ALMACENAMIENTO ---
        Artista       Género musical     Tipo  \
0   Billy Ocean  Latin (Origen: pop)  Canción   
1   Nicki Minaj  Latin (Origen: pop)  Canción   
2  David Guetta  Latin (Origen: pop)  Canción   
3  Joey Montana  Latin (Origen: pop)  Canción   
4       Pitbull  Latin (Origen: pop)  Canción   

                                      Nombre  Año de lanzamiento  \
0  Caribbean Queen (No More Love On the Run)                2010   
1                                 Super Bass                2010   
2                  Memories (feat. Kid Cudi)                2010   
3                                 La Melodía                2010   
4                   Guantanamera (She's Hot)                2010   

               ID_Spotify                ID_Album  Popularidad  \
0  4JEylZNW8SbO4zUyfVrpb7  7n4OT3zEZaEiyKKd6mFAhi           75   
1  3hlksXnvbKogFdPbpO9vel  7aADdYLiK1z7GlMFr0UIZw           80 

In [21]:
# --- CELDA 6: FUSIÓN DE SPOTIFY (CARGADO DE CSV) + LAST.FM Y GUARDADO FINAL ---

import pandas as pd
import os

print("\n--- INICIANDO FUSIÓN DE DATOS (SPOTIFY + LAST.FM) ---")

# 1. CARGA DEL CSV INICIAL DE SPOTIFY
# CRÍTICO: Debe ser el nombre del archivo guardado en la Celda 3 (o la Celda 4, si la ejecutaste).
spotify_input_filename = 'latin_data_2010_2024.csv' 
df_spotify_cargado = None # Inicializamos la variable

if os.path.exists(spotify_input_filename):
    df_spotify_cargado = pd.read_csv(spotify_input_filename)
    print(f"✅ DataFrame de Spotify cargado desde CSV: {len(df_spotify_cargado)} filas.")
else:
    print(f"❌ ERROR: No se encuentra el archivo CSV inicial de Spotify: {spotify_input_filename}")


if df_spotify_cargado is not None and 'df_lastfm' in locals():
    
    # 2. Realizamos la fusión (Merge) usando df_spotify_cargado como base
    df_proyecto_final = pd.merge(
        df_spotify_cargado, 
        df_lastfm,
        on='Artista', 
        how='left'
    )
    
   
    # Rellenar valores nulos con 0 y asegurar que sean enteros
    for col in ['Playcount_LastFM', 'Listeners_LastFM']:
        if col in df_proyecto_final.columns:
            # Primero rellenamos NaN con 0.0 (float) y luego convertimos a int
            df_proyecto_final[col] = df_proyecto_final[col].fillna(0).astype(int)
    
    
    # 4. Guardar el archivo CSV FINAL
    output_filename = 'data_latinpop.csv'
    
    df_proyecto_final.to_csv(
        output_filename, 
        index=False, 
        encoding='utf-8'
    )
    
    print(f"\n🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉")
    print(f"Total de registros finales: {len(df_proyecto_final)}")
    print("--- Primeras 5 filas con datos clave ---")
    print(f"✅ Archivo final guardado: {output_filename}")
    
else:
    print("❌ ERROR DE FUSIÓN: Asegúrate de que la Celda 5 (Last.fm) se haya ejecutado exitosamente para crear 'df_lastfm'.")


--- INICIANDO FUSIÓN DE DATOS (SPOTIFY + LAST.FM) ---
✅ DataFrame de Spotify cargado desde CSV: 500 filas.

🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉
Total de registros finales: 500
--- Primeras 5 filas con datos clave ---
✅ Archivo final guardado: data_latinpop.csv


In [1]:
!pip install mysql-connector-python
# O usa el que suele ser más robusto con Pandas:
!pip install sqlalchemy mysqlclient

In [2]:
# --- CELDA 1 CORREGIDA: CREACIÓN Y CONEXIÓN A MYSQL ---
# --- CELDA 1: CONFIGURACIÓN Y CONEXIÓN A MYSQL (VERIFICACIÓN DE IMPORTACIONES) ---

import pandas as pd
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, BigInteger, Text, VARCHAR # <--- ¡ESTAS!
from sqlalchemy.sql import text # <--- ¡ESTA!
import mysql.connector 
import numpy as np # <--- ¡ESTA!
# ... (el resto de tu código de conexión sigue aquí)

# 1. Cargar las credenciales y definir el motor de conexión
load_dotenv('2.env') 

CSV_FILENAME = 'latin_data_2010_2024.csv'

DB_USER = os.getenv("MYSQL_USER")
DB_PASSWORD = os.getenv("MYSQL_PASSWORD")
DB_HOST = os.getenv("MYSQL_HOST")
DB_DATABASE = os.getenv("MYSQL_DATABASE")
engine = None # Inicializamos la variable

try:
    # --- PASO 1: CONECTARSE AL SERVIDOR Y CREAR LA BASE DE DATOS ---
    
    # 1.1 Conexión sin especificar la base de datos (Database=None)
    conn_no_db = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cursor = conn_no_db.cursor()
    
    # 1.2 Crear la base de datos si no existe
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_DATABASE}")
    cursor.close()
    conn_no_db.close()
    
    print(f"✅ Base de datos '{DB_DATABASE}' creada o verificada en el servidor.")
    
    
    # --- PASO 2: CREAR EL ENGINE FINAL CON LA BASE DE DATOS ESPECIFICADA ---
    
    mysql_url = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_DATABASE}"
    engine = create_engine(mysql_url)
    
    print(f"✅ Conexión establecida a la base de datos MySQL: {DB_DATABASE}")

    # 1.3 Cargar el DataFrame de la Fase 1
    df_raw = pd.read_csv(CSV_FILENAME)
    df_raw = df_raw.rename(columns={'Año de lanzamiento': 'Año_lanzamiento'}) 
    
    print(f"✅ Datos cargados y columna renombrada: {len(df_raw)} filas.")
    print("\n--- ¡LISTO PARA EL MODELADO EN MYSQL! ---")

except ImportError:
    print("❌ ERROR: Asegúrate de que instalaste 'mysql-connector-python' o 'mysqlclient'.")
except Exception as e:
    print(f"❌ ERROR CRÍTICO DE CONEXIÓN: Verifica que tu servidor MySQL esté encendido y tus credenciales en '2.env' sean correctas. Detalle: {e}")
    engine = None

✅ Base de datos 'latinpop' creada o verificada en el servidor.
✅ Conexión establecida a la base de datos MySQL: latinpop
✅ Datos cargados y columna renombrada: 500 filas.

--- ¡LISTO PARA EL MODELADO EN MYSQL! ---


In [3]:
# --- CELDA 2 CORREGIDA: INSERCIÓN EN MYSQL (USANDO TIPOS DE SQLALCHEMY) ---

if 'engine' not in locals() or engine is None:
    print("❌ ERROR: El motor de conexión a MySQL no se creó correctamente en la Celda 1.")
else:
    print("--- INICIANDO INSERCIÓN DE DATOS EN MYSQL ---")
    
    # 1. Preparar la tabla ARTISTAS
    df_artistas = df_raw[[
        'Artista', 
        'Playcount_LastFM', 
        'Listeners_LastFM', 
        'Biografia_Resumen'
    ]].drop_duplicates(subset=['Artista']).dropna(subset=['Artista'])
    
    df_artistas = df_artistas.replace({np.nan: None}) 

    # Insertar en MySQL. Usamos los objetos de tipo SQLAlchemy.
    df_artistas.to_sql('ARTISTAS', engine, if_exists='replace', index=False, dtype={
        # Usamos VARCHAR para el nombre del artista, especificando la longitud
        'Artista': VARCHAR(255),
        'Playcount_LastFM': BigInteger, # Usamos BigInteger para números grandes
        'Listeners_LastFM': BigInteger,
        'Biografia_Resumen': Text
    },
    # Añadimos una instrucción CREATE TABLE manual para la Clave Primaria (PK)
    # SQLAlchemy no maneja las PK complejas de esta forma en to_sql, así que se la inyectamos:
    schema=DB_DATABASE,
    chunksize=100)
    
    # Después de crear la tabla, ejecutamos el ALTER TABLE para añadir la PK
    with engine.connect() as connection:
        connection.execute(text("ALTER TABLE ARTISTAS ADD PRIMARY KEY (Artista)"))
        connection.commit()
    
    print(f"✅ Insertados {len(df_artistas)} artistas únicos en la tabla ARTISTAS.")

    # 2. Preparar la tabla ALBUMES
    df_albumes = df_raw[[
        'ID_Album', 
        'Nombre_Album', 
        'Año_Lanzamiento_Album', 
        'Artista_Principal_Album'
    ]].drop_duplicates(subset=['ID_Album']).dropna(subset=['ID_Album'])

    df_albumes = df_albumes.replace({np.nan: None}) 
    
    # Insertar en MySQL
    df_albumes.to_sql('ALBUMES', engine, if_exists='replace', index=False, dtype={
        'ID_Album': VARCHAR(50),
        'Nombre_Album': VARCHAR(255),
        'Año_Lanzamiento_Album': Integer,
        'Artista_Principal_Album': VARCHAR(255),
    }, chunksize=100)
    
    with engine.connect() as connection:
        connection.execute(text("ALTER TABLE ALBUMES ADD PRIMARY KEY (ID_Album)"))
        connection.commit()
    
    print(f"✅ Insertados {len(df_albumes)} álbumes únicos en la tabla ALBUMES.")


    # 3. Preparamos la tabla CANCIONES
    df_canciones = df_raw[[
        'ID_Spotify', 
        'Nombre', 
        'Año_lanzamiento', 
        'ID_Album'
    ]].drop_duplicates(subset=['ID_Spotify']).dropna(subset=['ID_Spotify'])

    df_canciones = df_canciones.replace({np.nan: None}) 

    # Insertar en MySQL
    df_canciones.to_sql('CANCIONES', engine, if_exists='replace', index=False, dtype={
        'ID_Spotify': VARCHAR(50),
        'Nombre': VARCHAR(255),
        'Año_lanzamiento': Integer,
        'ID_Album': VARCHAR(50)
    }, chunksize=100)
    
    with engine.connect() as connection:
        connection.execute(text("ALTER TABLE CANCIONES ADD PRIMARY KEY (ID_Spotify)"))
        connection.commit()
        
    print(f"✅ Insertadas {len(df_canciones)} canciones únicas en la tabla CANCIONES.")

    # 4. Nota: Puedes añadir aquí las restricciones de CLAVE FORÁNEA (FOREIGN KEY) si lo deseas.
    # Por ahora, solo nos aseguramos de que los datos estén en la BBDD.
    
    print("\n--- FASE 2 (MODELADO) COMPLETADA. Datos insertados en MySQL. ---")

--- INICIANDO INSERCIÓN DE DATOS EN MYSQL ---


C:\Users\USUARIX\AppData\Local\Temp\ipykernel_9804\3701334701.py:19: UserWarning: The provided table name 'ARTISTAS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_artistas.to_sql('ARTISTAS', engine, if_exists='replace', index=False, dtype={


✅ Insertados 254 artistas únicos en la tabla ARTISTAS.


C:\Users\USUARIX\AppData\Local\Temp\ipykernel_9804\3701334701.py:49: UserWarning: The provided table name 'ALBUMES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_albumes.to_sql('ALBUMES', engine, if_exists='replace', index=False, dtype={


✅ Insertados 337 álbumes únicos en la tabla ALBUMES.
✅ Insertadas 500 canciones únicas en la tabla CANCIONES.

--- FASE 2 (MODELADO) COMPLETADA. Datos insertados en MySQL. ---


C:\Users\USUARIX\AppData\Local\Temp\ipykernel_9804\3701334701.py:74: UserWarning: The provided table name 'CANCIONES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_canciones.to_sql('CANCIONES', engine, if_exists='replace', index=False, dtype={


In [4]:
# --- PLANTILLA DE ANÁLISIS ---

# Asegúrate de que el 'engine' está disponible desde la Celda 1.
if 'engine' not in locals() or engine is None:
    print("❌ ERROR: El motor de conexión a MySQL no está disponible.")
else:
    try:
        # 1. DEFINE AQUÍ TU NUEVA CONSULTA SQL
        query_mi_nuevo_analisis = """
        SELECT 
            T1.Columna_A,
            COUNT(T1.Columna_B) 
        FROM 
            TuTabla AS T1
        -- Usa INNER JOIN para conectar tus tablas (CANCIONES, ALBUMES, ARTISTAS)
        INNER JOIN ...
        WHERE 
            ...
        GROUP BY 
            ...
        """
        
        # 2. EJECUTA Y CONVIERTE EL RESULTADO A DATAFRAME
        df_nuevo_analisis = pd.read_sql_query(text(query_mi_nuevo_analisis), engine)
        
        print("--- RESULTADO DEL NUEVO ANÁLISIS ---")
        print(df_nuevo_analisis.to_string(index=False))
        
    except Exception as e:
        print(f"❌ ERROR al ejecutar la consulta SQL: {e}")

❌ ERROR al ejecutar la consulta SQL: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '...
        WHERE 
            ...
        GROUP BY 
            ...' at line 7
[SQL: 
        SELECT 
            T1.Columna_A,
            COUNT(T1.Columna_B) 
        FROM 
            TuTabla AS T1
        -- Usa INNER JOIN para conectar tus tablas (CANCIONES, ALBUMES, ARTISTAS)
        INNER JOIN ...
        WHERE 
            ...
        GROUP BY 
            ...
        ]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [5]:
# --- FASE 3: ANÁLISIS 1 - POPULARIDAD PROMEDIO POR AÑO ---

import pandas as pd
from sqlalchemy.sql import text # Necesario para ejecutar la consulta

# Asegúrate de que el 'engine' está disponible desde la Celda 1.

if 'engine' not in locals() or engine is None:
    print("❌ ERROR: El motor de conexión a MySQL no está disponible. Ejecuta la Celda 1 de la Fase 2.")
else:
    try:
        # Consulta SQL: Popularidad Promedio por Año
        # Unimos las tres tablas para obtener las métricas de LastFM (en ARTISTAS)
        # y agruparlas por el año de lanzamiento de la canción (en CANCIONES).
        
        query_popularidad_anual = """
        SELECT 
            T1.Año_lanzamiento AS Año,
            AVG(T3.Listeners_LastFM) AS Promedio_Listeners,
            AVG(T3.Playcount_LastFM) AS Promedio_Playcount,
            COUNT(T1.ID_Spotify) AS Total_Canciones_Muestra
        FROM 
            CANCIONES AS T1
        
        -- Conexión 1: De CANCIONES a ALBUMES (por ID_Album)
        INNER JOIN
            ALBUMES AS T2 ON T1.ID_Album = T2.ID_Album
        
        -- Conexión 2: De ALBUMES a ARTISTAS (por Artista_Principal_Album)
        INNER JOIN
            ARTISTAS AS T3 ON T2.Artista_Principal_Album = T3.Artista
        
        GROUP BY 
            T1.Año_lanzamiento
        ORDER BY 
            T1.Año_lanzamiento;
        """
        
        # Ejecutar la consulta y cargar resultados en un DataFrame
        df_popularidad_anual = pd.read_sql_query(text(query_popularidad_anual), engine)
        
        print("--- RESULTADO DEL ANÁLISIS DE POPULARIDAD POR AÑO (2010 vs 2024) ---")
        print(df_popularidad_anual.to_string(index=False)) # Imprime la tabla sin el índice
        
    except Exception as e:
        print(f"❌ ERROR al ejecutar la consulta SQL: {e}")

--- RESULTADO DEL ANÁLISIS DE POPULARIDAD POR AÑO (2010 vs 2024) ---
 Año  Promedio_Listeners  Promedio_Playcount  Total_Canciones_Muestra
2010        1123879.4167        4.356592e+07                      240
2024         467970.0950        2.556193e+07                      242
